In [1]:
import bs4
import openai
import io
import os
import dotenv

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory
from langchain.storage import LocalFileStore
from langchain_community.document_loaders import WebBaseLoader, DirectoryLoader, TextLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import CacheBackedEmbeddings
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.document_loaders import PyPDFLoader

In [2]:
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Load, chunk and index the contents of the blog.
loader = PyPDFLoader("data/writegreatcode_vol2.pdf")
docs = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model="gpt-4", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, try to answer the question."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

#create question and answer chain and combine chains

qa_system_prompt = """You are an assistant for question-answering tasks. \
If you don't know the answer, just say that you don't know. \

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [8]:
def ask_question(question):
    ai_msg = rag_chain.invoke({"input": question, "chat_history": chat_history})
    print(ai_msg["answer"])

In [3]:
chat_history = []

ask_question("what is this pdf about?")

This PDF appears to be about a book titled "Write Great Code: Thinking Low-Level, Writing High-Level" by Randall Hyde. The book focuses on the aspect of writing efficient and high-performance code. It is not about optimization, but rather about understanding how to write code that performs well. The book is part of the Write Great Code series.


In [4]:
ask_question("give me some takeaways")

1. Premature optimization can lead to problems in software design. It's not usually worth spending a lot of time micro-optimizing code before it's clear where the performance bottlenecks are.

2. However, performance issues should always be considered from the beginning when designing software at a system level.

3. Great code has several attributes: it uses the CPU, memory, and system resources efficiently; it is easy to read and maintain; and it follows a consistent set of style guidelines.

4. No matter how efficient your code is, if it is not readable and maintainable by others, then it’s not great code. 

5. Writing great code involves a personal level of craftsmanship, art, and pride in workmanship. 

6. Coding styles, commenting, code layout, and other coding tasks contribute to making code readable and easy to maintain.


In [9]:
ask_question("list me the chapters in this pdf.")
#some chapters are missing from the pdf

The chapters in this PDF are:

1. Chapter 5: Compiler Operation and Code Generation
2. Chapter 6: Tools for Analyzing Compiler Output
3. Chapter 7: Constants and High-Level Languages
4. Chapter 8: Variables in a High-Level Language
5. Chapter 9: Array Data Types
6. Chapter 10: String Data Types
7. Chapter 11: Pointer Data Types
8. Chapter 12: Record, Union, and Class Data Types
9. Chapter 13: Arithmetic and Logical Expressions
10. Chapter 14: Control Structures and Programmatic Decisions
11. Chapter 15: Iterative Control Structures
12. Chapter 16: Functions and Procedures

There are also sections on Engineering Software, an Appendix comparing the 80x86 and PowerPC CPU Families, and Online Appendices.


In [10]:
ask_question("what about chapters 1-4?")
#unfortunately, the model does not know the answer...

The text doesn't provide specific information about the content of chapters 1-4. However, it mentions that chapters 3 and 4 provide a quick primer for 80x86 and PowerPC assembly language. The content of chapters 1 and 2 is not specified in the provided text.
